In [19]:
import os
import json
from openai import OpenAI
import instructor
from pydantic import BaseModel, Field
from typing import List
import numpy as np
from utils import get_structured_feedback, remove_duplicates, Evaluation, CriteraFeedback

In [20]:
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

instructor_client = instructor.from_openai(openai_client)

In [21]:
remove_duplicates(openai_client=openai_client, strings=['Clear and logical organization of the essay', 'Clear and logical organization'])

['Clear and logical organization of the essay']

In [22]:
remove_duplicates(openai_client=openai_client, strings=['i am good', 'i am bad'])

['i am good']

In [4]:
# read messages from user1-messages.json
with open("user1-messages.json", "r") as f:
	chat_history = json.load(f)

In [5]:
feedbacks = get_structured_feedback(instructor_client, chat_history)

### Combining evaluations from a chat session to remove duplicates

In [6]:
def aggregate_feedback(feedbacks: List[Evaluation]):
    # Initialize lists for each feedback category
    aggregated = {
        "introduction": {"strengths": [], "weaknesses": [], "suggestions": []},
        "structure": {"strengths": [], "weaknesses": [], "suggestions": []},
        "argumentation": {"strengths": [], "weaknesses": [], "suggestions": []},
        "evidence": {"strengths": [], "weaknesses": [], "suggestions": []},
        "conclusion": {"strengths": [], "weaknesses": [], "suggestions": []}
    }
    
    # Aggregate all feedbacks
    for feedback in feedbacks:
        for key in aggregated:
            critera_feedback = getattr(feedback, key)
            aggregated[key]['strengths'].extend(critera_feedback.strengths)
            aggregated[key]['weaknesses'].extend(critera_feedback.weaknesses)
            aggregated[key]['suggestions'].extend(critera_feedback.suggestions)
    
    # Deduplicate lists
    for key in aggregated:
        for subkey in aggregated[key]:
            aggregated[key][subkey] = remove_duplicates(openai_client, aggregated[key][subkey])

    # Construct a new Evaluation object
    return Evaluation(
        introduction=CriteraFeedback(**aggregated['introduction']),
        structure=CriteraFeedback(**aggregated['structure']),
        argumentation=CriteraFeedback(**aggregated['argumentation']),
        evidence=CriteraFeedback(**aggregated['evidence']),
        conclusion=CriteraFeedback(**aggregated['conclusion'])
    )

In [7]:
aggregated_feedback = aggregate_feedback(feedbacks)

/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/

In [8]:
for criteria, feedback in aggregated_feedback:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Effectively frames the argument and makes the stance clear and compelling']
Weaknesses: []
Suggestions: []


structure:
Strengths: ['Clear and logical structuring of the essay']
Weaknesses: ['Consider developing the counterargument section with more details or statistics about economic development']
Suggestions: ['More depth in certain areas']


argumentation:
Strengths: ['Use of examples like protecting natural habitats and the impact on global warming']
Weaknesses: ['Lack of depth in the counterargument section']
Suggestions: ['Consider developing the counterargument section with more details or statistics about economic development']


evidence:
Strengths: ['Use of examples to solidify the argument and make the essay more persuasive']
Weaknesses: []
Suggestions: []


conclusion:
Strengths: ['Effectively restating the thesis and summarizing main points']
Weaknesses: []
Suggestions: ['More depth in certain areas']




In [9]:
# read messages from user1-messages.json
with open("user2-messages.json", "r") as f:
	chat_history_user2 = json.load(f)

In [10]:
feedbacks2 = get_structured_feedback(instructor_client, chat_history_user2)

In [11]:
aggregated_feedback2 = aggregate_feedback(feedbacks2)

In [13]:
for criteria, feedback in aggregated_feedback2:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Relatable and engaging language used to discuss environmental conservation']
Weaknesses: ['Lack of factual information and statistics to strengthen the argument']
Suggestions: ['Incorporate more factual information and statistics for a persuasive argument']


structure:
Strengths: []
Weaknesses: ['Lack of clear structure with introductions, body paragraphs, and conclusions']
Suggestions: ['Structure the essay with clear introductions, body paragraphs, and conclusions for improved flow and readability']


argumentation:
Strengths: []
Weaknesses: []
Suggestions: ['Incorporate research to make the argument more persuasive']


evidence:
Strengths: []
Weaknesses: []
Suggestions: []


conclusion:
Strengths: []
Weaknesses: []
Suggestions: []




In [14]:
all_aggregated_feedbacks = [aggregated_feedback, aggregated_feedback2]

In [16]:
res = aggregate_feedback(all_aggregated_feedbacks)

/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/

In [17]:
for criteria, feedback in res:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Effectively frames the argument and makes the stance clear and compelling']
Weaknesses: ['Lack of factual information and statistics to strengthen the argument']
Suggestions: ['Incorporate more factual information and statistics for a persuasive argument']


structure:
Strengths: ['Clear and logical structuring of the essay']
Weaknesses: ['Consider developing the counterargument section with more details or statistics about economic development']
Suggestions: ['More depth in certain areas']


argumentation:
Strengths: ['Use of examples like protecting natural habitats and the impact on global warming']
Weaknesses: ['Lack of depth in the counterargument section']
Suggestions: ['Consider developing the counterargument section with more details or statistics about economic development']


evidence:
Strengths: ['Use of examples to solidify the argument and make the essay more persuasive']
Weaknesses: []
Suggestions: []


conclusion:
Strengths: ['Effectively resta

In [18]:
remove_duplicates(openai_client, ['i am good', 'i am bad'])

['i am good']